In [1]:
# test_imports in this cell, restart kernel whenever edits to modules are made

import sys
import os

# Append the src directory to the path
current_dir = os.getcwd()
src_dir = os.path.abspath(os.path.join(current_dir, '..', 'src'))
if src_dir not in sys.path:
    sys.path.append(src_dir)

#import config based on which machine the jupyter notebook is running on
from configs.config_local import *
from preprocessing import *
from hub_calls import *
from postprocessing import *
from utils import *

def test_imports():
    print("testing config class")
    config = Config()
    print(config)

    print("testing 2d contacts preprocessing classes")
    query = HiCQuery(config)
    print(query)

    print("testing helper function imports")
    out = ValidLoop(config, 10000)
    print(out)
    
    #other prints for imported classes

test_imports()

testing config class
testing 2d contacts preprocessing classes
HiC file loaded
testing helper function imports


In [4]:
####### Loading Config object #######

config = Config()
query = HiCQuery(config)
# check if hic file is readable
inform = query.read_hic_header(config.paths.hic_file)
print(inform)
current_chrom = config.genomic_params.chromosomes[0]
current_res = config.genomic_params.resolutions[0]
currest_res0 = config.genomic_params.res_strs[0]

HiC file loaded
{'version': '8', 'master_index': '3126283484', 'genome_id': '/var/lib/cwl/stga3f80174-ba8e-4d25-8491-09d0707bbd92/4DNFI823LSII.chrom.sizes', 'attributes': {b's': b'o'}}


In [5]:
######## Query o/e from .hic as csr matrix ########

contacts_oe = query.oe_intra(current_chrom, current_res)
print(contacts_oe[0:10])
sparse_matrix_oe = query.straw_to_csr(contacts_oe, current_res)
print(sparse_matrix_oe.data[0:10])

#save for quick access later
sparse_matrix_path = os.path.join(config.paths.temp_dir, f"/{currest_res0}/sparse_matrix_{current_chrom}.npz")
np.savez_compressed(sparse_matrix_path, data=sparse_matrix_oe.data, indices=sparse_matrix_oe.indices, indptr=sparse_matrix_oe.indptr, shape=sparse_matrix_oe.shape)
print(f"Sparse matrix saved at {sparse_matrix_path}")

Filename: /Users/Akanksha/MaGroup/Genomic Hubs/3D-genome-hubs/src/preprocessing/_2Dcontacts_processing.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    94   1677.5 MiB   1677.5 MiB           1       @profile
    95                                             def oe_intra(self, chrom, res):
    96                                                 """
    97                                                 returns csr sparse matrix of contact records for one chromosome
    98                                                 straw object : .binX [0] .binY [1] .counts [2] as attributes
    99                                                 """
   100   1677.5 MiB      0.0 MiB           1           chrom = chrom[3:]
   101   1677.5 MiB      0.0 MiB           1           res = int(res)
   102   2554.6 MiB    877.2 MiB           2           oe_list = hicstraw.straw(
   103   1677.5 MiB      0.0 MiB           1               "oe", self.hic_norm, self.hic_file, chrom, chrom, "B

- [ ]  ######### Global contacts; loose clusters/compartments ########
For OE: Call A/B compartments at 1Mb (GT only available for 4DN not ENCODE)
    - [ ]  Call a/b comps from csr matrix, save as bigwig as bin_annotations
    - [ ]  Reliability of a/b score:
        - [ ]  viz oe for a specific region from hic and bigwig track for alignment of a/b calls
        - [ ]  plot correlation between GT a/b scores and called a/b scores
    - [ ]  Creating graph object:
        - [ ]  1. threshold O/E for node-set creation and visualise the effect of thresholding on the matrix
        - [ ]  2. save node-sets for three different thresholds, each o/e edge weight is 1
    - [ ]  Spectral clustering:
        - [ ]  1. save clusters as sparse matrix for each threshold, viz clusters and color by a/b score of each bin
    - [ ]  Reliability of clustering:
        - [ ]  plot confusion matrix between clustered nodes and GT a/b scores
    - [ ]  ### then parallelize the above steps for all chromosomes ###

In [ ]:
#### Global interactions {Large scale: 1Mb-500Kb resolution} ####
